In [2]:
import os


import pprint
import random
import numpy as np
import pandas as pd

import spacy
import sys
sys.path.append("../")
from datatools.analyzer import *
# from datatools.analyzer import clean_text
from error_tools import *

In [3]:
from nltk.lm        import Vocabulary
# from nltk.lm.models import MLE
from nltk.lm.models import KneserNeyInterpolated
from nltk.util      import ngrams

In [4]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [5]:
conv_utt = []
for conv in convs:
    for i, ut in enumerate(conv):
        if not ut.is_exist_error():
            conv_utt.append(clean_text(ut.utt))

In [18]:
def load_utt_ntt():
    ntt_path = "../../corpus/NTT/"
    utt_list = []
    for file_ in os.listdir(ntt_path):
        if not "json" in file_:
            continue 
        with open(ntt_path+file_, "r",  encoding="utf-8") as f:
            convs = json.load(f)
            for did in convs["convs"]:
                dids = list( did.keys() )[0]
                conv = did[dids]
                # utt_list.extend( [ clean_text(utt)  for utt in conv])
                utt_list.extend( [utt  for utt in conv])
    print(len(utt_list))
    return utt_list

In [19]:
ntt_utt = load_utt_ntt()

141777


In [26]:
ntt_span_utt = [str(doc) for doc in sentence2docs(ntt_utt)]

In [6]:
def read_file(filename, batch_size=10000) -> list:
    f = open(filename, 'r', encoding='UTF-8')
    text_list = [  ]
    concated = ""
    for i, line in enumerate( f.readlines() ):
        concated += line.rstrip('\n')
        if (i+1) % batch_size == 0:
            text_list.append(concated)
            concated = ""
    text_list.append(concated)
    f.close()
    return text_list

In [7]:
filename = "../../corpus/wiki/wiki_40b_train_normal.txt"
all_list = read_file(filename, batch_size=100)
print("len of text_list :", len(all_list))


len of text_list : 18674


In [36]:
all_wiki = []
for line in tqdm(all_list):
    all_wiki += [sen+"。" for sen in line.split("。")]


100%|██████████| 18674/18674 [00:02<00:00, 8556.43it/s]


In [37]:
mecab_analyzer = Analyzer(tokenizer_)
# 一文ずつしか送られて来ない読み
def sentence2normalize_noun_mecab(sentences):
    normalize_sen = []
    for sen in sentences:
        df = mecab_analyzer.analyze_with_dataframe(sen)
        words = []
        for txt, pos in zip(df.surface, df.part_of_speech):
            # pos_split = pos.split("-")
            # if pos_split[0]
            if "名詞" in pos:
                words.append(pos)
            else:
                words.append(txt)
        normalize_sen.append(words)
    return normalize_sen

In [38]:
all_utt = ntt_span_utt + all_wiki[::100]

In [39]:
filled_normal = fill_SYMBOL_ONE( sentence2normalize_noun_mecab(all_utt) )

In [40]:
def create_language_model(sentences, N):
    vocab = Vocabulary([word for sent in sentences for word in sent])
    text_ngrams = [ngrams(sent, N) for sent in sentences]
    lm = KneserNeyInterpolated(order=N, vocabulary=vocab)
    lm.fit(text_ngrams)
    return lm

In [41]:
n=3
lm = create_language_model(filled_normal, N=n)

In [42]:
from datatools.maneger import DataManager

In [43]:
modelM = DataManager("../models/utterance/")
# model_name = "KLM_phrase_n={0}.pickle".format(n)
# model_name = "KLM_phrase_nucc_n={0}.pickle".format(n)
# model_name = "KLM_phrase_nucc_n={0}_orth.pickle".format(n)
model_name = "KLM_phrase_nucc_wiki_n={0}_noun1.pickle".format(n)

In [44]:
modelM.save_data(model_name, lm)

success save : ../models/utterance/KLM_phrase_nucc_wiki_n=3_noun1.pickle
